In [1]:
from bs4 import BeautifulSoup
import base64
import wget
import requests
import unicodecsv as csv
import datetime
import dateutil


In [2]:
total_news = [] # it will save all news in a array

In [3]:
## get days
iterator_day = datetime.date(2019, 9, 22)#iterator day
init_date = datetime.date(2019, 1, 1) #final day of scrapping

In [4]:
#array with days of scrapping
array_days = [str(iterator_day)]
while(iterator_day >= init_date):
    iterator_day = iterator_day - datetime.timedelta(days=1)
    array_days.append(str(iterator_day))

In [5]:
def get_title(content_soup):
    if content_soup.find('h1', {'class': 'news-title'}) is not None:
        return content_soup.find('h1', {'class': 'news-title'}).text
    
    return;

In [6]:
def get_summary(content_soup):
    if content_soup.find('h2', {'class': 'news-summary'}) is not None:
        return content_soup.find('h2', {'class': 'news-summary'}).text
    
    return;

In [7]:
def get_author(content_soup):
    if content_soup.find('span', {'class': 'author-name'}).find('a') is not None:
        return content_soup.find('span', {'class': 'author-name'}).find('a').text
    return;

In [8]:
def get_date(content_soup):
    if content_soup.find('time', {'class': 'news-date'}).text is not None:
        return content_soup.find('time', {'class': 'news-date'}).text
    
    return;

In [9]:
def get_content(content_soup):
    if content_soup.find('p', {'class': 'first-parrafo'}) is not None:
        return content_soup.findAll('p', {'class': 'first-parrafo'})
    return [];

In [10]:
#save a string array in a csv file
def save_csv(articles):
    with open('el_trome.csv', 'wb') as f:
            dict_writer = csv.DictWriter(
                f,
                [
                    'Titulo',
                    'Autor',
                    'Resumen',
                    'Contenido', 
                    'Categoria', 
                    'Fecha', 
                ],
                encoding='utf-8')
            dict_writer.writeheader()
            dict_writer.writerows(articles)

In [14]:
def interate_articles_and_save(articles):
    cont = 0
    for article in articles:
        print('-------------------- Article ----------------------')
        id_article = article.find('div', {'class': 'flow-detail'}).find('h2',{'class': 'flow-title'}).find('a',{'class': 'page-link'})
        url_article ='https://trome.pe'+ id_article['href']
        content_article = requests.get(url_article).content
        soup_article = BeautifulSoup(content_article,'lxml')
        print("URL Article===>" + str(url_article))
        
        
        title = get_title(soup_article)
        print(title)
        summary = get_summary(soup_article)
        print(summary)
        category = article.find('div', {'class': 'flow-detail'}).find('h3',{'class': 'flow-category'}).find('a').find('span').text
        print(category)
     
        author = get_author(soup_article)
        print(author)
        string_date = get_date(soup_article)
        if string_date is not None:
            string_date =string_date[0:(string_date.find('h'))].replace(" - ",' ')+":00"
            date = dateutil.parser.parse(string_date)
        else:
            date =""
        print(str(date))
        text_content = get_content(soup_article)
        content_article = []
        for textito in text_content:    
            content_article.append(textito.text)
        print(content_article)
        total_news.extend([
            {
                'Titulo':str(title), 
                'Autor':str(author),
                'Resumen': str(summary), 
                'Contenido':str(content_article),   
                'Categoria':str(category), 
                'Fecha':date,
            }
        ])
        save_csv(total_news)
        cont = cont + 1
        print('-----------------------------------  ' + str(cont*100/len(articles)) + "%")
        print('-------------------- End Article ----------------------')
    

In [15]:
def iterate_publications(days):
    for day in days:
        print('-------------------- Day ----------------------')
        url_day = 'https://trome.pe/todas-las-noticias/todas/' + day
        content_day = requests.get(url_day).content
        soup_day = BeautifulSoup(content_day,'lxml')
        print("URL Day===>" + str(url_day))
        articles_day = soup_day.findAll('article',{'class': 'section-flow'})
        try:
            interate_articles_and_save(articles_day)
        except:
            print('Error para obtener pagina====> ' + url_day)
        print('-------------------- End Day ----------------------')

In [ ]:
iterate_publications(array_days)

-------------------- Day ----------------------
URL Day===>https://trome.pe/todas-las-noticias/todas/2019-09-03
-------------------- Article ----------------------
URL Article===>https://trome.pe/celebrities/harry-styles-volvio-cortarse-cabello-genera-diversas-reacciones-fans-nndc-134514
Harry Styles volvió a cortarse el cabello y genera diversas reacciones en sus fans
El cantante fue captado en Italia con un nuevo look que generó todo tipo de comentarios en sus fanáticos.  
Celebrities 
Redacción Trome  
2019-03-09 23:05:00
['El cantante y actor Harry Styles  volvió a sorprender a sus fans con un nuevo corte de cabello, tal como se aprecia en unas fotos publicadas por la cuenta de Instagram “thisharrystyles”.\xa0', 'En la publicación,  el exintegrante de One Direction aparece mirando a la cámara de unos fanáticos y luce con el cabello mucho más corto.\xa0', 'Según la cuenta, la foto fue tomada hace poco en Italia, aunque no fue la única que muestra al intérprete junto a un fanático.\x

URL Article===>https://trome.pe/celebrities/termino-mal-relacion-paulina-rubio-gerardo-bazua-conoce-historia-nnda-nnlt-134505
¿Por qué terminó tan mal la relación entre Paulina Rubio y Gerardo Bazúa? Conoce su historia
El cantante de música ranchera Gerardo Bazúa denunció en redes sociales a Paulina Rubio por no dejarla ver a su hijo. Conoce esta relación que acabó en malos términos 
Celebrities 
Redacción Trome  
2019-03-09 21:33:00
['La guerra se ha desatado entre Paulina Rubio   y su expareja  Gerardo Bazúa  , padre del menor de los hijos de la cantante, luego que denunciara a la Chica Dorada de dar órdenes a sus empleados para que le impidan ver a su retoño. ', 'Esta situación ha provocado que el también cantante de música ranchera  acuse a la artista de evitar que su pequeño pase tiempo con su familia paterna. Por tal motivo, a través de su cuenta de Instagram, acusó a Rubio de no llegar a un acuerdo sobre el régimen de visitas de Eros porque, según él, no le resultaría convenient

URL Article===>https://trome.pe/celebrities/friends-sofa-central-perk-exhibira-cuatro-paises-latinoamerica-mexico-argentina-colombia-brasil-nndc-134493
“Friends”: El sofá de Central Perk se exhibirá en cuatro países de Latinoamérica
Warner Bros llevará el icónico sofá naranja a distintos lugares, incluidas capitales de Latinoamérica, como parte de las celebraciones del 25 aniversario de  la exitosa serie.
Celebrities 
Redacción Trome  
2019-03-09 19:06:00
['Se cumplen 25 años desde el estreno de “Friends” y las celebraciones continúan. Warner Bros hará una gira por diversas ciudades del mundo con uno de los elementos más recordados de la serie.\xa0', 'Se trata del sofá naranja de Central Perk, donde se vivieron innumerables escenas desde el primer episodio y que estará disponible en varias ciudades de Estados Unidos y de otros países, incluidas  cuatro\xa0 latinoamericanas.', 'Según USA Today,  serán 30 las réplicas que aparecerán en diversos locales del mundo, incluyendo lugares al ai

URL Article===>https://trome.pe/celebrities/liam-payne-defiende-justin-bieber-cuestiona-medios-comunicacion-nndc-134489
Liam Payne defiende a Justin Bieber y cuestiona a los medios de comunicación
Liam Payne criticó el tratamiento que le dieron algunos medios de comunicación a las recientes revelaciones del intérprete de "Baby".  
Celebrities 
Redacción Trome  
2019-03-09 18:31:00
[' Justin Bieber  se robó la atención de todos tras publicar en Instagram un extenso mensaje donde  confesó que consumió drogas desde los 19 años. Como era de esperarse, la noticia dio la vuelta al mundo. ', 'Sin embargo, a Liam Payne, la exestrella de One Direction, no le agradó el tratamiento que le dieron algunos medios de comunicación a las recientes revelaciones del intérprete de “Baby”.', 'Liam compartió una captura de una publicación que hizo un medio, en la que se lee el siguiente titular:  "Justin Bieber admite el uso excesivo de drogas y sus problemas con la fama en una publicación sincera".  ', ' “

URL Article===>https://trome.pe/mundo/youtube-viral-desalmado-sujeto-patea-bebe-ano-china-camara-seguridad-grabo-cruel-agresion-134388
Desalmado sujeto pateó a bebé de un año en China y cámara de seguridad registró la cruel agresión | VIDEO
Cámara de centro comercial grabó el momento, en el que un desadaptado pateó a una pequeña de un año, en  China.  Autoridades locales ya buscan a este sujeto. 
Mundo 
Redacción Trome  
2019-02-09 22:45:00
['Un lamentable hecho ha despertado el repudio general de gran cantidad de ciudadanos en China. Y es que una cámara de seguridad de un centro comercial grabó los precisos momentos en los que un sujeto pateó a una pequeña de un año que se encontraba al cuidado de su abuela en un centro comercial. Las autoridades de China ya buscan al desalmado individuo.', 'Según las primeras informaciones, el hecho ocurrió el pasado 28 de agosto en un centro comercial de la ciudad de Shenzhen. En las imágenes se puede apreciar como este sujeto camina por detrás de l

...

URL Article===>https://trome.pe/mundo/mexico/mexico-detienen-mujer-acusada-asesinar-cuchillo-bebe-video-134262
México: Detienen a mujer acusada de asesinar con un cuchillo a su bebé | VIDEO
Una mujer de 20 años fue arrestada en México  , sindicada de haber acuchillado a su bebé dentro de su vivienda,. 
México 
Redacción Trome  
2019-01-09 21:13:00
["Gran repudió ha causado en México, el cruel asesinato de un bebé, presuntamente a manos de su propia madre, quien ya fue detenida por la policía. El sangriento episodio se presentó en el estado de Nuevo León, en el municipio de Santa Catarina.Según vecinos de la colonia Zimix, ellos escucharon un horrendo grito de una vivienda y al ingresar encontraron a Melany 'N', de 20 años, en medio de un charco de sangre. De inmediato la auxiliaron y la llevaron a un centro médico, donde los galenos diagnosticaron que la mujer había dado a luz.\xa0 ", "Ante esta situación, la policía llegó hasta la vivienda donde encontraron el cuerpo del bebé. Este pr

URL Article===>https://trome.pe/espectaculos/tv/primo-don-ramon-historia-don-roman-familiar-olvidado-vecindad-chavo-8-nnda-nnlt-134260
El primo de Don Ramón: conoce la historia de Don Román, el familiar olvidado de la vecindad
Don Román, aquel personaje conocido por ser el primo de Don Ramón, hizo su aparición en dos capítulos de El Chavo del 8. Conoce su historia y qué fue de él 
Tv 
Redacción Trome  
2019-01-09 20:28:00
['Los personajes de El Chavo del 8   marcaron la infancia de muchas personas. ¿Quién no recuerda a ese niño que se escondía en un barril y le gustaba las tortas de jamón?, ¿o a aquella niña pecosa que culpaba de sus travesuras a los demás?, ¿o de repente a ese chiquillo mimado y a veces egoísta que paraba con los cachetes inflados?, ¿o a esa mujer que llamaba chusma a sus vecinos?, ¿o quizás a la que era confundida con una bruja por su aspecto?, ¿también al señor que cobraba la renta?, ¿o al profesor que iba a la vecindad a ver al amor de su vida?, ¿e incluso a ese ho

URL Article===>https://trome.pe/mundo/francia-torero-peruano-joaquin-galdos-recibio-cornada-grave-muslo-derecho-tuvo-intervenido-video-nndc-rt-134251
Torero peruano Joaquín Galdós recibió cornada en el muslo derecho y su pronóstico es "muy grave" | VIDEO
En una feria taurina en Bayona, el matador peruano Joaquín Galdós fue embestido con violencia por un toro y voló por los aires.  
Mundo 
Redacción Trome  
2019-01-09 19:39:00
['El torero peruano Joaquín Galdós quedó gravemente herido este domingo luego de ser embestido violentamente por un toro con una cornada durante una corrida que se realizó en la plaza francesa de Bayona. ', ' Joaquín Galdós recibió fuertes cornadas y voló por los aires cuando intentaba dar el estoque de muerte al sexto toro durante la Feria del Atlántico de Bayona.', 'El peruano\xa0Joaquín Galdós de 23 años tuvo que ser auxiliado por banderilleros y otros toreros para lograr salir de la arena.', 'Tras el ataque,  Joaquín Galdós fue trasladado al Hospital Belarra d

URL Article===>https://trome.pe/deportes/alianza-lima-vs-deportivo-municipal-vivo-ver-golperu-tv-gratis-online-trujillo-torneo-clausura-liga-1-134177
Alianza Lima empató 2-2 con Deportivo Municipal y perdió chance de tomar punta del Clausura | VIDEO | FOTOS
 Alianza Lima ganaba 2-0 a  Deportivo Municipal y los ediles, con dos penales, emparejaron la cuenta en el estadio Mansiche de Trujillo por la fecha 5 del Torneo Clausura.  
Deportes 
Redacción Trome  
2019-01-09 18:34:00
[" Alianza Lima tuvo la posibilidad de encontrar por fin su norte y la punta del Torneo Clausura, pero\xa0 Deportivo Municipal \xa0interrumpió el sueño blanquiazul que en el estadio de Estadio Mansiche sorprendían con un marcador a favor y terminaron cediendo un empate 2-2 sobre los minutos finales.\xa0Tuvo  Alianza Lima la iniciativa de llevarse los tres puntos desde los primeros minutos con incursiones de Federico Rodríguez, durante todo el partido asediando el área de los barrenderos e incluso se encargó de gene

URL Article===>https://trome.pe/deportes/tabla-posiciones-torneo-clausura-tabla-acumulada-fecha-5-liga-1-134069
Tabla de posiciones del Clausura tras el empate de Alianza Lima con Municipal por fecha 5 de la Liga 1
 Tabla de posiciones: Se disputa la fecha 5 del Torneo Clausura  de la  Liga 1 
Deportes 
Redacción Trome  
2019-01-09 17:59:00
['Tabla de posiciones:  Se juega la fecha 5 del Torneo Clausura   de la Liga 1. Sport Huancayo es el líder de la clasificación y comparte el primer lugar con Universitario, Unión Comercio y Alianza Lima. ', 'Alianza Lima empató con Municipal, cedió dos puntos y desperdició la oportunidad de quedar en solitario como líder de la tabla de posiciones. ', 'Universitario venció a Melgar y trepó a los primeros lugar de la tabla del Clausura. Los cremas son colíderes del torneo con 10 puntos, el mismo puntaje que Sport Huancayo y Unión Comercio, pero por diferencia de goles se ubica tercero. ', ' Sporting Cristal \xa0superó a César Vallejo en Trujillo. Los 